In [1]:
import pyodbc
import requests
from bs4 import BeautifulSoup
from datetime import date
import datetime
import time
import pandas as pd
import random
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import os

In [ ]:
"""
This is a program to pull data from Yahoo finance to save it to your server and analyze averages
1. Need to create a table with stock list named "stock_list"
2. "stock_list" needs columns "Stock_Symbol", "Stock_Type", "Stock_Name", "Buy_Price" (optional), "Sold_Price" (optional), Note (optional)
3. Error might occur, if there are too many stocks, so run it again, if error occurs for no reason.
4. All data will be saved back to the server
"""

In [2]:
#settings
today = str (date.today())
print(today)
n_repeat = 90
#locations that you want results to be saved
path = r"Reports\%sReport.xlsx" % today

#sql connection
#server that you are pulling the data from
conn = ('DRIVER={SQL Server};SERVER=DESKTOP-IDDUKHI\SQLEXPRESS; DATABASE=YahooDB;Trusted_Connection=yes')
quoted = quote_plus(conn)
new_con = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(new_con)
sql_conn = pyodbc.connect(conn)

2019-06-14


In [3]:
def timer(counter):
    if counter % 30 == 0 or counter % 25 == 0:
        print('waiting')
        time.sleep(0.002)
        
def ran_timer():
    time.sleep(random.random()/random.randrange(10,100))

def df_add(df, seq, i, last_date, last_closed):
    #adding data into column
    new_df = [seq, i, last_date, last_closed] 
    print(new_df)
    df.loc[-1] = new_df
    #fixing index
    df.index = df.index + 1 

def df_sort(df):
    df = df.sort_values(by = ['SEQ'], ascending = False, axis = 0)
    return df

    
def mine_data(i, finder):
    url = "https://finance.yahoo.com/quote/"+i+"/history"
    headers = {'User-Agent': 'Mozilla/5.0'}
    page = requests.get(url,headers=headers)
    bsobj = BeautifulSoup(page.content, "html.parser")
    data = bsobj.find("span",{"data-reactid":"%s" %finder})
    if data == None:
        data = None
    else:
        data = ' '.join([i.strip() for i in data])
        data = data.replace(",", "")
    return data

def int_date(date):
    date = str(date.replace(',',''))
    return int(datetime.datetime.strptime(date, "%b %d %Y").strftime('%Y%m%d'))

In [4]:
class finder:
    def __init__(self, loc):
        self.loc = loc
        self.name = self
        
    def add(self, x):
        self.loc += x
        
    #next day for normal stocks
    def n_s_n(self):
        x = 15
        self.add(x)
        
    #next day for normal funds
    def n_f_n(self):
        y = 14
        self.add(y)
    
    #next day for stocks with dividend 
    def d_s_n(self):
        z = 22
        self.add(z)
        
    #next day for funds with dividend 
    def d_f_n(self):
        k = 21
        self.add(k)
    
    def empty_date(self):
        x = -1
        self.add(x)

In [5]:
#SQL
def checkTableExists(dbcon, tablename):
    dbcur = dbcon.cursor()
    dbcur.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(tablename.replace('\'', '\'\'')))
    if dbcur.fetchone()[0] == 1:
        dbcur.close()
        return True

    dbcur.close()
    return False

In [6]:
def new_data(n_repeat, i, engine):
    #creating a new file and dataframe
    print('New Stock: %s'%i)
    df_n = pd.DataFrame(columns = ['SEQ', 'Stock Name', 'Date', 'Closed Price'])
    
    #setting
    last_date_finder = finder(51)
    last_closed_finder = finder(61)
    second_date_finder = finder(66)

    n_repeat = n_repeat
    counter = 0
    fund_finder = False
    
    #Data minning
    while n_repeat > 0:
        counter += 1
        if counter %30 == 0 or counter %25 ==0:
            print('waiting')
            timer(counter)
            print(counter)
        else:
            print(counter)   
        
        last_date = mine_data(i, last_date_finder.loc)
        seq = int_date(last_date)
        
        second_date = mine_data(i, second_date_finder.loc)
        if second_date == None:
            print('Second date empty')
            ran_timer()
            print('Funds')
            # moving it to next date
            if fund_finder == False:
                second_date_finder.empty_date()
            fund_finder = True
            print(second_date_finder.loc)
            second_date = mine_data(i, second_date_finder.loc)
        seq_s = int_date(second_date)  
        
        #normal setting
        if seq != seq_s:
            last_closed = mine_data(i, last_closed_finder.loc)
            df_add(df_n, seq, i, last_date, last_closed)
            ran_timer()
            n_repeat -= 1
            if fund_finder == True:
                last_date_finder.n_f_n()
                last_closed_finder.n_f_n()
                second_date_finder.n_f_n()
            else:
                last_date_finder.n_s_n()
                last_closed_finder.n_s_n()
                second_date_finder.n_s_n()
                
        elif seq == seq_s:
            print('Dividend')
            last_closed = mine_data(i, last_closed_finder.loc)
            df_add(df_n, seq, i, last_date, last_closed)
            n_repeat -= 1
            
            if fund_finder == True:
                last_date_finder.d_f_n()
                last_closed_finder.d_f_n()
                second_date_finder.d_f_n()
            else:
                last_date_finder.d_s_n()
                last_closed_finder.d_s_n()
                second_date_finder.d_s_n()
        else:
            print('Error')      
    df_n.to_sql(name = i, con = engine, if_exists = 'fail')

In [7]:
def old_data(n_repeat, sql_conn, i, engine):
    #reading an exisiting file
    print('Updating existing stock: %s'%i)
    query_1 = 'SELECT MAX([SEQ]) as SEQ FROM [%s]' %i
    df_o = pd.read_sql(query_1, sql_conn)
    last_date_o = df_o['SEQ'][0]
    print('last date: %s' %last_date_o) 
    query_2 = 'SELECT * FROM [{0}] WHERE [SEQ] = {1}'.format(i, last_date_o)
    df = pd.read_sql(query_2, sql_conn)
    df = df[['SEQ', 'Stock Name', 'Date', 'Closed Price']]

    #setting
    last_date_finder = finder(51)
    last_closed_finder = finder(61)
    second_date_finder = finder(66)
    counter = 0
    fund_finder = False
    seq_counter = 0
    seq_o = last_date_o
    
    last_date = mine_data(i, last_date_finder.loc)
    seq = int_date(last_date)
    second_date = mine_data(i, second_date_finder.loc)
    if second_date == None:
        seq_s = int_date(mine_data(i, second_date_finder.loc - 1))
    else:
        seq_s = int_date(second_date)
    
    if seq-seq_o > 80:
        print("data missing")
        ran_timer()
        print('The current table is outdated.')
        new_data(n_repeat, i, engine)
        
    else:
        
        while seq_o != seq:
            last_date = mine_data(i, last_date_finder.loc)
            seq = int_date(last_date)
            second_date = mine_data(i, second_date_finder.loc)

            #stocks with dividend
            if second_date == None:
                print('second date empty')
                ran_timer()
                second_date = mine_data(i, second_date_finder.loc)
                print('funds')
                second_date_finder.empty_date()
                fund_finder = True
                second_date = mine_data(i, second_date_finder.loc)

            first_closed_finder = finder(61)
            first_closed = mine_data(i, first_closed_finder.loc)

            if first_closed == None:
                print('Dividend for wmt')
                ran_timer()
                last_date_finder = finder(58)
                last_closed_finder = finder(66)
                second_date_finder = finder(73)
                second_date = mine_data(i, second_date_finder.loc)
                last_closed = mine_data(i, last_date_finder.loc)

            seq_s = int_date(second_date)

            #normal stocks
            if seq != seq_s:
                last_closed = mine_data(i, last_closed_finder.loc)
                if first_closed != None:
                    
                    df_add(df, seq, i, last_date, last_closed)
                    df = df_sort(df)
                    
                    seq_counter += 1
                    seq_o = df['SEQ'][seq_counter]
                    
                    ran_timer()
                    
                    if fund_finder == True:
                        last_date_finder.n_f_n()
                        last_closed_finder.n_f_n()
                        second_date_finder.n_f_n()
                    else:
                        last_date_finder.n_s_n()
                        last_closed_finder.n_s_n()
                        second_date_finder.n_s_n()

            elif seq == seq_s:
                print(seq)
                print(seq_s)
                print('Dividend')
                last_closed = mine_data(i, last_closed_finder.loc)
                
                df_add(df, seq, i, last_date, last_closed)
                df_sort(df)
                
                seq_counter += 1
                seq_o = df['SEQ'][seq_counter]
                
                last_date_finder.d_s_n()
                last_closed_finder.d_s_n()
                second_date_finder.d_s_n()
        df = df.drop_duplicates(subset = ['SEQ'], keep = "last")
        df = df[:-1]
        print(df)
        df.to_sql(name = i, con = engine, if_exists = 'append')

In [8]:
def analyze_data(df, stock_list, columns, sql_conn):
    for i in stock_list:
        query_1 = 'SELECT [SEQ], [Stock Name], [Date], [Closed Price] FROM [%s]' %i
        df_o = pd.read_sql(query_1, sql_conn)
        df_o['Closed Price'] = df_o['Closed Price'].astype(float)
        df_f = df
        current_seq = df_o.loc[0,"SEQ"]
        current_price = df_o.loc[0,"Closed Price"]
        # creating averages
        days = list(range(10,91,10))
        aver ={}
        for day in days:
            aver[day] = 1-(df_o.loc[0:(day-1),"Closed Price"].mean()/current_price)
        df_f.loc[len(df_f)]=[current_seq,stock_dic_type[i],i,stock_dic[i], 
                             note_dic[i], current_price, buy_dic[i], 
                             (current_price - buy_dic[i])/ buy_dic[i],
                             aver[10],aver[20],aver[30],aver[40],aver[50],aver[60],aver[70],aver[80],aver[90]]
    return df_f

In [9]:
def organize_report(df_f, path, stock_list):
    #formating data
    df_f = df_f.sort_values(by=['Stock_Type', '10A'])
    print(df_f)
    path = path
    writer = pd.ExcelWriter(path, engine='xlsxwriter')
    df_f.to_excel(writer, sheet_name='Sheet1')
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    format_data = workbook.add_format({'num_format': '0.00%'})
    worksheet.set_column('I2:Z99', None, format_data)

    format1 = workbook.add_format({'bg_color':   '#FFC7CE','font_color': '#9C0006'})
    worksheet.conditional_format('I2:Z99', {'type': 'cell', 'criteria': '<', 'value': 0, 'format': format1})

    worksheet.set_column('E:E', 23)

    format2 = workbook.add_format({'bg_color':   '#FFFF00'})
    worksheet.conditional_format('C1:C999', {'type': 'text', 'criteria': 'containing', 'value' : 'Index', 'format': format2})

    format3 = workbook.add_format({'bg_color':   '#008000'})
    worksheet.conditional_format('C1:C999', {'type': 'text', 'criteria': 'containing', 'value' : 'Owned', 'format': format3})

    writer.save()

    #open a file

    os.startfile(path)

In [10]:
#changing a table to lists
query_1 = "SELECT * FROM [YahooDB].[dbo].[stock_list]"
df1 = pd.read_sql(query_1, sql_conn)
stock_list = df1["Stock_Symbol"]
stock_names = df1["Stock_Name"]
stock_dic = dict(zip(stock_list, stock_names))
stock_type = df1["Stock_Type"]
stock_dic_type = dict(zip(stock_list,stock_type))
buy_price = df1["Buy_Price"]
buy_dic = dict(zip(stock_list, buy_price))
note = df1["Note"]
note_dic = dict(zip(stock_list, note))

In [11]:
# excuting functions
for i in stock_list:
#     stock_table = '[dbo].%s' %i
    if checkTableExists(sql_conn, i) == False:
        #need safe net, if data fails
        new_data(n_repeat, i, engine)
    else:
        old_data(n_repeat, sql_conn, i, engine)

print('Analyzing all stock data')
columns = ['Date', 'Stock_Type', 'Stock_Symbol', 'Stock_Name', 'Note', 'Current Price', 'Buy Price', 'Gain / Loss',  '10A', '20A', '30A','40A','50A','60A','70A','80A','90A']        
df_f = pd.DataFrame(columns = columns)
df_f = analyze_data(df_f, stock_list, columns, sql_conn)
    
organize_report(df_f, path, stock_list)


Updating existing stock: ^DJI
last date: 20190614
Empty DataFrame
Columns: [SEQ, Stock Name, Date, Closed Price]
Index: []
Updating existing stock: ^IXIC
last date: 20190614
Empty DataFrame
Columns: [SEQ, Stock Name, Date, Closed Price]
Index: []
Updating existing stock: ^GSPC
last date: 20190614
Empty DataFrame
Columns: [SEQ, Stock Name, Date, Closed Price]
Index: []
Updating existing stock: GE
last date: 20190614
Empty DataFrame
Columns: [SEQ, Stock Name, Date, Closed Price]
Index: []
Updating existing stock: BEARX
last date: 20190613
Empty DataFrame
Columns: [SEQ, Stock Name, Date, Closed Price]
Index: []
Updating existing stock: FSAGX
last date: 20190613
Empty DataFrame
Columns: [SEQ, Stock Name, Date, Closed Price]
Index: []
Updating existing stock: FTQGX
last date: 20190613
Empty DataFrame
Columns: [SEQ, Stock Name, Date, Closed Price]
Index: []
Updating existing stock: WMT
last date: 20190614
Empty DataFrame
Columns: [SEQ, Stock Name, Date, Closed Price]
Index: []
Updating exist